In [1]:
-- DATA
require 'cunn'
X = torch.zeros(5000, 2)
Y = torch.zeros(5000)

for i = 1, 5000 do
    X[i] = (torch.rand(2,1) - 0.5 ) * 20
    Y[i] = X[i][1]^2 + X[i][1]*X[i][2]+X[i][2]^2
end

X = X:float():cuda()
Y = Y:float():cuda()

In [2]:
-- split as train set and test set
torch.manualSeed(0)
Ind = torch.randperm(5000)

trainInd = Ind[ { {   1, 4500} } ]
testInd  = Ind[ { {4501, 5000} } ]

train = { data  = X:index(1, trainInd:long()),
          label = Y:index(1, trainInd:long())}

tra_x1 = train.data[{ {},{1} }]:clone():resize(4500)
tra_x2 = train.data[{ {},{2} }]:clone():resize(4500)
tra_y = train.label:clone():resize(4500)

test  = { data  = X:index(1, testInd:long()),
          label = Y:index(1, testInd:long())}

tes_x1 = test.data[{ {},{1} }]:clone():resize(500)
tes_x2 = test.data[{ {},{2} }]:clone():resize(500)
tes_y = test.label:clone():resize(500)

In [3]:
-- construct neural network
require 'nn'

nInput = 2
nOutput = 1
nHidden = 10
nHid2 = 5
nHid3 = 5

net = nn.Sequential()
net:add(nn.Linear(nInput, nHidden))      -- fully connected 
net:add(nn.ReLU())                   -- non-linearity
net:add(nn.Linear(nHidden, nHid2))    -- fully connected
net:add(nn.ReLU())                   -- non-linearity
-- net:add(nn.Linear(nHid2, nHid3))    -- fully connected
-- net:add(nn.ReLU())                   -- non-linearity
net:add(nn.Linear(nHid2, nOutput))    -- fully connected
net = net:cuda()                         -- GPU

-- define loss: MSE
criterion = nn.MSECriterion()
-- criterion.sizeAverage = false
criterion = criterion:cuda()

In [4]:
params, gradParams = net:getParameters()

In [5]:
require 'optim'

-- log test loss and train loss
logger = optim.Logger('acc.log')
logger:setNames{'Trainn acc .', 'Test acc .'}

-- batch gd
local optimState = {learningRate = 0.0001}

batchSize = 4500
maxEpoch = 2000
nBatch = maxEpoch * 4500/batchSize

n = 1
for k = 1, maxEpoch do
    for i = 1, 4500, batchSize do
        -- define batch inputs
        Batch = {  data = train.data[{ {i, i+batchSize-1} }],
                  label = train.label[{ {i, i+batchSize-1} }] }
        
        -- define feval function for optim.sgd
        function feval(params)
            gradParams:zero()
        
            local outputs = net:forward(Batch.data)
            local loss = criterion:forward(outputs, Batch.label)
            local dloss_doutputs = criterion:backward(outputs, Batch.label)
            net:backward(Batch.data, dloss_doutputs)
            
            return loss, gradParams
        end
        
        -- train use batch sgd
        newParams, train_loss = optim.sgd(feval, params, optimState)
        
        -- check convergence
        if newParams:ne(newParams):sum() > 0 then
            print(n, 'not converge')
        end
        
        -- test current network
        pred = net:forward(test.data)
        test_loss = criterion:forward(pred, test.label)
        
        -- store loss
        logger:add{train_loss[1], test_loss}
        
        n = n + 1
    end
end

In [6]:
logger:style{'-', '-'}
logger:plot()

In [7]:
-- plot train
gnuplot.scatter3( tra_x1, tra_x2, tra_y)

In [10]:
-- Data Visualization
x = torch.linspace(-20,20, 100)
y = torch.zeros(100, 100)
for i = 1, 100 do
    for j = 1, 100 do
        y[i][j] = x[i]^2 + x[i]*x[j]+x[j]^2
    end
end

gnuplot.splot(y)

In [9]:
-- plot test and pred
gnuplot.scatter3( {tes_x1, tes_x2, pred[1]:resize(500)}, {tes_x1, tes_x2, tes_y} )

In [10]:
gnuplot.scatter3( {tes_x1, tes_x2, pred[1]:resize(500)})